## Import packages:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk 
import gensim
import re
import datetime
import collections
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt

%matplotlib inline

## Load dataset:

In [2]:
train = pd.read_csv("./Dataset/cleaned_data.csv", delimiter="\t")

In [3]:
train.head()

,Unnamed: 0,body
0,FM20160429032ACC9MUHD,acea essere pronto giocare partita banda ultra...
1,FM20160501023ACLAzwID,unico certezza momento essere prossimo tappa s...
2,FM20160503035ACaNSzJD,potere aspettare mese luglio formalizzazione p...
3,FM20160503035ACDx00JD,fondo chiuso aiutare piccolo medio impresa ita...
4,FM20160428033ACi2kvGD,volto provocazione sortire effetto sperare ext...


In [4]:
train.shape

(9283, 2)

In [5]:
def tagDocuments(text):
    for i, line in enumerate(text):
        if(i%1000 == 0):
            print("> Iteration: " + str(i))
        yield gensim.models.doc2vec.TaggedDocument(
            [w for w in 
             gensim.utils.simple_preprocess(line)], [i])

## Extract train data from dataframe into list:

In [6]:
print("> START %s" % datetime.datetime.now())
train_corpus = list(tagDocuments(train['body'].tolist()))
print("> END %s" % str(datetime.datetime.now()))

len(train_corpus)

> START 2017-05-21 22:33:36.598067
> Iteration: 0
> Iteration: 1000
> Iteration: 2000
> Iteration: 3000
> Iteration: 4000
> Iteration: 5000
> Iteration: 6000
> Iteration: 7000
> Iteration: 8000
> Iteration: 9000
> END 2017-05-21 22:33:38.696122


9283

## Prepare model:

In [7]:
window = 20
size = 20
dm_concat = 1
alpha = 0.02
model = gensim.models.doc2vec.Doc2Vec(size=size, min_count=10, dm_concat= dm_concat, window=window, alpha = alpha,iter=55)

In [17]:
model_name = "doc2vec_21_05_2017"
model.save(model_name)

In [8]:
model.build_vocab(train_corpus)

## Train model:

In [9]:
%time model.train(train_corpus)

CPU times: user 23min 39s, sys: 3.1 s, total: 23min 42s
Wall time: 8min 7s


57535414

In [10]:
ranks = [] #per ogni documento l'indice che indica a che livello si trova nella lista dei simili a se stesso)

second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

## Evaluate model (raw):

In [11]:
def score(listOfRanks):
    correct = 0
    for e in listOfRanks:
        if(e >= 0 | e <= 2):
        #if( e == 0):
            correct = correct + 1
    return correct/len(listOfRanks)

In [12]:
score(ranks)

0.9593881288376602

## Model inspection

In [13]:
model.infer_vector(['oggi', 'voglio', 'comprare', 'una', 'azione', 'di', 'mediaset'])

array([ 0.07834738,  0.09632134,  0.10528717, -0.04028289,  0.06526494,
       -0.16741337, -0.08751242, -0.03783806, -0.00122402,  0.34268799,
       -0.13431677,  0.10227473,  0.0537323 , -0.13337226, -0.11831887,
       -0.15145037,  0.0541449 ,  0.03916775, -0.00842663,  0.06210634], dtype=float32)

In [14]:
model.wv.vocab['gruppo'].count

9215

### NOTE:

#### ranks:
    - num_elements = #num_docs
    - per ogni documento ( in ordine) abbiamo l'indice che rappresenta dove trovare se stesso nella lista dei suoi simili
    - particolarità: il documento 9282 è il 184 documento più simile a se stesso
        
#### second_ranks:
    - num_elements = #num_docs
    - per ogni domcumento ( in ordine) abbiamo il secondo documento più simile

In [15]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (9282): «toscano aeroporto apprendere indiscrezione stampa commissione valutazione impatto ambientale ministero ambiente avere esprimere parere favorevole masterplan aeroporto amerigo vespucci firenze prevedere altro cosa realizzazione cosiddetto pista parallelo nuovo terminal aeroportuale quotare mercato telematico azionario organizzare gestire borsa italiano riservare qualsiasi commento merito fino pubblicazione parere relativo documentazione parte ministero riferire nota»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d20,n5,w20,mc10,s0.001,t3):

MOST (9282, 0.8507392406463623): «toscano aeroporto apprendere indiscrezione stampa commissione valutazione impatto ambientale ministero ambiente avere esprimere parere favorevole masterplan aeroporto amerigo vespucci firenze prevedere altro cosa realizzazione cosiddetto pista parallelo nuovo terminal aeroportuale quotare mercato telematico azionario organizzare gestire borsa italiano riservare qualsiasi commento merito fino pubbl

In [16]:
# Pick a random document from the test corpus and infer a vector from the model

doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (4305): «gruppo campari apprestare lanciare opa amichevole grand marnier secondo riportare figaro erede jean baptiste lapostolle louis alexandre marnier controllare maggioranza capitale grand marnier diritto voto avere accettare cedere quota gruppo italiano secondo quotidiano francese transazione valorizzare grand marnier oltre miliardo euro doppio mercato milione campari fondare trattare acquisizione importante storia azienda interpellare indiscrezione stampa avere commentare»

Similar Document (4306, 0.8570370078086853): «gruppo campari apprestare lanciare opa amichevole grand marnier secondo riportare figaro erede jean baptiste lapostolle louis alexandre marnier controllare maggioranza capitale grand marnier diritto voto avere accettare cedere quota gruppo italiano secondo quotidiano francese transazione valorizzare grand marnier oltre miliardo euro doppio mercato milione campari fondare trattare acquisizione importante storia azienda interpellare indiscrezione stampa